# Real Estate Price Prediction Project

## Importing the necessary bibliotheca

In [51]:
import numpy as np
import pandas as pd
from sklearn.svm import SVR  # Importiere den SVM-Regressor
import pickle
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

## Load the Data

In [52]:
#data = pd.read_csv('belgian_property_prices.csv')
data = pd.read_csv('../raw_data/belgian_property_prices.csv')
#data = pd.read_csv('raw_data/belgian_property_prices.csv')
data.head()

,as_built_plan,available_as_of,basement,bathrooms,bedroom_1_surface,bedroom_2_surface,bedroom_3_surface,bedrooms,building_condition,co2_emission,...,width_of_the_lot_on_the_street,yearly_theoretical_total_energy_consumption,housenumber,street,city,postal,state,lat,lng,price
0,0.0,After signing the deed,1.0,1.0,12.0,10.0,10.0,3.0,Good,9802.0,...,11.0,NaN,30,Stationsstraat,Ronse,9600.0,Vlaams Gewest,50.743367,3.601306,275000
1,0.0,After signing the deed,1.0,1.0,15.0,11.0,11.0,4.0,Good,NaN,...,NaN,NaN,52,Wingenesteenweg,Wingene,8750.0,Vlaams Gewest,51.042146,3.302600,430000
2,NaN,After signing the deed,1.0,3.0,22.0,22.0,22.0,5.0,As new,NaN,...,6.0,NaN,157,Mechelsesteenweg,Antwerpen,2018.0,Vlaams Gewest,51.205030,4.410943,949000
3,0.0,NaN,NaN,3.0,33.0,30.0,23.0,5.0,As new,NaN,...,19.0,NaN,66,Noordzandstraat,Brugge,8000.0,Vlaams Gewest,51.206826,3.219186,2200000
4,0.0,After signing the deed,NaN,NaN,19.0,NaN,NaN,1.0,Just renovated,58.0,...,5.0,24263.0,15,Rue de l'Etoile,Wavre,1301.0,Région Wallonne,50.717062,4.577550,195000


## Exploratory Data Analysis, EDA

In [53]:
data.info()

data.describe()

data.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4770 entries, 0 to 4769
Data columns (total 54 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   as_built_plan                                   2389 non-null   float64
 1   available_as_of                                 3912 non-null   object 
 2   basement                                        2083 non-null   float64
 3   bathrooms                                       4523 non-null   float64
 4   bedroom_1_surface                               3163 non-null   float64
 5   bedroom_2_surface                               3105 non-null   float64
 6   bedroom_3_surface                               2623 non-null   float64
 7   bedrooms                                        4740 non-null   float64
 8   building_condition                              4485 non-null   object 
 9   co2_emission                             

Index(['as_built_plan', 'available_as_of', 'basement', 'bathrooms',
       'bedroom_1_surface', 'bedroom_2_surface', 'bedroom_3_surface',
       'bedrooms', 'building_condition', 'co2_emission', 'cadastral_income',
       'connection_to_sewer_network', 'construction_year',
       'covered_parking_spaces', 'dining_room', 'double_glazing',
       'energy_class', 'external_reference', 'flood_zone_type', 'furnished',
       'garden_surface', 'gas_water__electricity', 'heating_type',
       'kitchen_surface', 'kitchen_type', 'latest_land_use_designation',
       'living_area', 'living_room_surface', 'number_of_frontages', 'office',
       'outdoor_parking_spaces', 'planning_permission_obtained',
       'possible_priority_purchase_right', 'primary_energy_consumption',
       'proceedings_for_breach_of_planning_regulations',
       'reference_number_of_the_epc_report', 'street_frontage_width',
       'subdivision_permit', 'surface_of_the_plot', 'surroundings_type',
       'tv_cable', 'tenemen

## Preprocessing

In [54]:
import pandas as pd

data = pd.read_csv('../raw_data/belgian_property_prices.csv')

data_raw_backup = data.copy()

data_raw_backup.to_csv('belgian_property_prices_backup.csv', index=False)

data.head()

,as_built_plan,available_as_of,basement,bathrooms,bedroom_1_surface,bedroom_2_surface,bedroom_3_surface,bedrooms,building_condition,co2_emission,...,width_of_the_lot_on_the_street,yearly_theoretical_total_energy_consumption,housenumber,street,city,postal,state,lat,lng,price
0,0.0,After signing the deed,1.0,1.0,12.0,10.0,10.0,3.0,Good,9802.0,...,11.0,NaN,30,Stationsstraat,Ronse,9600.0,Vlaams Gewest,50.743367,3.601306,275000
1,0.0,After signing the deed,1.0,1.0,15.0,11.0,11.0,4.0,Good,NaN,...,NaN,NaN,52,Wingenesteenweg,Wingene,8750.0,Vlaams Gewest,51.042146,3.302600,430000
2,NaN,After signing the deed,1.0,3.0,22.0,22.0,22.0,5.0,As new,NaN,...,6.0,NaN,157,Mechelsesteenweg,Antwerpen,2018.0,Vlaams Gewest,51.205030,4.410943,949000
3,0.0,NaN,NaN,3.0,33.0,30.0,23.0,5.0,As new,NaN,...,19.0,NaN,66,Noordzandstraat,Brugge,8000.0,Vlaams Gewest,51.206826,3.219186,2200000
4,0.0,After signing the deed,NaN,NaN,19.0,NaN,NaN,1.0,Just renovated,58.0,...,5.0,24263.0,15,Rue de l'Etoile,Wavre,1301.0,Région Wallonne,50.717062,4.577550,195000


In [55]:
# Replace missing values for specified columns with the median
for col in ['bedroom_1_surface', 'bedroom_2_surface', 'bedroom_3_surface', 'bathrooms', 'bedrooms']:
    data[col] = data[col].fillna(data[col].median())

In [56]:
# Convert categorical variables with a few well-defined categories into numerical form using One-Hot Encoding.
# This is particularly suitable for variables like building_condition, state, and possibly city if you don't want to differentiate cities too strongly.
data = pd.get_dummies(data, columns=['building_condition', 'state'], drop_first=True)

In [57]:
#Define the mapping for categories in 'building_condition'
if 'building_condition' in data.columns:
    building_condition_mapping = {
        'To restore': 1,
        'To renovate': 2,
        'To be done up': 3,
        'Just renovated': 4,
        'Good': 5,
        'As new': 6
    }
    data['building_condition_encoded'] = data['building_condition'].map(building_condition_mapping)
    
    # Ersetze fehlende Werte mit 0 ohne inplace
    data['building_condition_encoded'] = data['building_condition_encoded'].fillna(0)
else:
    print("Column 'building_condition' does not exist in the DataFrame.")

Column 'building_condition' does not exist in the DataFrame.


In [58]:
# Convert date information: If 'available_as_of' contains a date, convert it and extract year, month, or weekday. If it's a description like "After signing the deed," use a dummy feature instead.
data['available_as_of'] = data['available_as_of'].apply(lambda x: 1 if x == 'After signing the deed' else 0)

In [59]:
# Combine bedroom areas: Sum bedroom surfaces to create 'total_bedroom_surface' as a new feature for better model insights.
data['total_bedroom_surface'] = data[['bedroom_1_surface', 'bedroom_2_surface', 'bedroom_3_surface']].sum(axis=1)

In [60]:
# Since the scale of numerical features (e.g., living_area, co2_emission, surface_of_the_plot) varies, it's beneficial to perform standardization or normalization. This improves model performance and is particularly useful for linear regression.
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
numerical_cols = ['bedroom_1_surface', 'bedroom_2_surface', 'bedroom_3_surface', 'bathrooms', 'living_area', 'surface_of_the_plot']
data[numerical_cols] = scaler.fit_transform(data[numerical_cols])

In [61]:
# Identify and handle outliers: Check for outliers in key numerical variables like price, living_area, or surface_of_the_plot that may impact regression. Consider removing values more than three standard deviations from the mean.
data = data[(data['price'] < data['price'].mean() + 3 * data['price'].std())]

In [62]:
data['building_condition'] = 'Unknown'  # Set a default value

In [63]:
building_condition_mapping = {
    'To restore': 1,
    'To renovate': 2,
    'To be done up': 3,
    'Just renovated': 4,
    'Good': 5,
    'As new': 6,
    'Unknown': 0  # Default for unknown values
}

data['building_condition_encoded'] = data['building_condition'].map(building_condition_mapping)

## Setting target variables

In [64]:
# Define target and select features

# Define target variable
price = data['price']

# Encode 'available_as_of' as a binary variable
data['available_as_of_encoded'] = data['available_as_of'].apply(lambda x: 1 if x == 'After signing the deed' else 0)

# Create 'total_bedroom_surface' by summing bedroom surface columns
data['total_bedroom_surface'] = data[['bedroom_1_surface', 'bedroom_2_surface', 'bedroom_3_surface']].sum(axis=1)

# Handle missing values without using inplace=True to avoid FutureWarnings
data['construction_year'] = data['construction_year'].fillna(data['construction_year'].median())
data['building_age'] = 2023 - data['construction_year']

data['toilets'] = data['toilets'].fillna(data['toilets'].median())
data['co2_emission'] = data['co2_emission'].fillna(data['co2_emission'].median())

In [65]:
# Encode 'building_condition'
building_condition_mapping = {'To restore': 1, 'To renovate': 2, 'To be done up': 3, 'Just renovated': 4, 'Good': 5, 'As new': 6, 'Unknown': 0}
data['building_condition_encoded'] = data['building_condition'].fillna('Unknown').map(building_condition_mapping)

# Core features and one-hot encoding
X = pd.concat([
    data[['bedrooms', 'bathrooms', 'living_area', 'surface_of_the_plot', 'postal', 'building_condition_encoded',
          'available_as_of_encoded', 'total_bedroom_surface', 'building_age', 'toilets', 'co2_emission']],
    pd.get_dummies(data[['energy_class', 'kitchen_type', 'heating_type']])
], axis=1)

## SVM-Modell

In [82]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import uniform

# Define parameter distribution for RandomizedSearchCV
param_dist = {
    'svm__C': uniform(0.1, 10),         # Sample C between 0.1 and 10
    'svm__epsilon': uniform(0.01, 0.5), # Sample epsilon between 0.01 and 0.5
    'svm__kernel': ['linear', 'rbf', 'poly']  # Fixed choices for kernel
}

# Set up the pipeline
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('svm', SVR())
])

# Randomized search with cross-validation
random_search = RandomizedSearchCV(pipeline, param_dist, n_iter=50, cv=5, scoring='r2', random_state=42, n_jobs=-1, verbose=2)
random_search.fit(X_train, y_train)

# Output best parameters and model performance
print(f"Best parameters: {random_search.best_params_}")
print(f"Best R² score from Random Search: {random_search.best_score_}")

# Evaluate on the test set using the best model found by RandomizedSearchCV
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)

# Calculate error metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Test Set Mean Squared Error (MSE): {mse}")
print(f"Test Set Mean Absolute Error (MAE): {mae}")
print(f"Test Set R-squared (R²): {r2}")

Best parameters: {'svm__C': 10, 'svm__epsilon': 0.01, 'svm__kernel': 'linear'}
Best R² score from Grid Search: 0.09773838764533874
Test Set Mean Squared Error (MSE): 84601376458.92065
Test Set Mean Absolute Error (MAE): 184618.8806171309
Test Set R-squared (R²): 0.1584919691831288


In [83]:
# Export the pipeline as a pickle file (tnx to utkarsh)
#import pickle
#with open('../models/best_model.pkl', 'wb') as file:
#    pickle.dump(model, file)